In [2]:
import pandas as pd
import numpy as np

def random_date_generator(start_date, range_in_days,number_of_items):
    days_to_add = np.arange(0, range_in_days)
    random_date = np.datetime64(start_date) + np.random.choice(days_to_add,size=number_of_items)
    return random_date

types = list('ABCD')


number_of_items = 1000
range_in_days = 321

info_df = pd.DataFrame(
        {"fab_date": random_date_generator("2019-01-01", range_in_days,number_of_items),
         "type": np.random.choice(types,size=number_of_items)   
                }
        )




totaltime = 400
max_force=10.8
time = np.arange(totaltime)
force_load = np.linspace(start=0,stop=max_force,num=190)
force_load = np.append(force_load,[np.float64(10.8)]*20)
force_load = np.append(force_load,np.linspace(start=max_force,stop=0,num=190))
test_df = pd.DataFrame(dict(force_load=force_load,time=time))


In [161]:
import os

In [162]:
os.getcwd()

'/media/hyamanieu/data/Applications/bk_pn_examples'

In [165]:
import sqlite3

In [171]:
def create_deformation_curve(typ,breaks=False):
    E_dic = {"A" : 90,
         "B" : 60,
         "C" : 90,
         "D" : 60,}
    H_dic = {"A" : 32,
         "B" : 24,
         "C" : 31,
         "D" : 22,}
    
    Y_dic  = {"A" : 800,
         "B" : 600,
         "C" : 600,
         "D" : 300,}
    
    max_f_dic   = {"A" : 10.8,
         "B" : 7,
         "C" : 7,
         "D" : 4.3,}
    
    
    
    totaltime = 400
    max_disp=max_f_dic[typ]
    E_mod = E_dic[typ] * (np.random.random_sample()*0.1+1)
    Y_str = Y_dic[typ] * (np.random.random_sample()*0.1+1)
    H = H_dic[typ] * (np.random.random_sample()*0.1+1)
    breaks = np.random.choice([True,False],size=None,p=[0.05,0.95])
    
    
    time = np.arange(totaltime,step=4)
    displacement = np.linspace(start=0,stop=max_disp,num=sum(time<190))
    displacement = np.append(displacement,[np.float64(10.8)]*sum(np.multiply(time>=190,time<=210)))
    displacement = np.append(displacement,np.linspace(start=max_disp,stop=0,num=sum(time>210)))
    
    
    stress = [0]
    broken = [0]
    mm = .3
    
    
    w = 7
    
    for i in range(1,len(displacement)):
        
        l = displacement[i] - displacement[i-1]
        
        
        d = stress[-1] + E_mod*l + (np.random.random_sample()-0.5)*mm
        b = 0
        
        if (l>0) and (d > Y_str) or (w < 7):
            if breaks:
                w -=1
                
            if w >= 4:
                d = stress[-1] + H*l + (np.random.random_sample()-0.5)*mm            
        
            if w < 4 :
                d = stress[-1] - H*l + (np.random.random_sample()-0.5)*mm
            
                
        elif d < mm*1.1:
            d = (np.random.random_sample()-0.5)*mm
            
            
        if w<0:
            b = 1
            d = (np.random.random_sample()-0.5)*mm
        
        stress.append(d)
        
    
    test_df = pd.DataFrame(dict(displacement=displacement,time=time,stress=stress))
    return test_df, E_mod, Y_str, H, breaks

In [172]:
import hvplot.pandas

In [173]:
test_df, E_mod, Y_str, H, breaks = create_deformation_curve("A",breaks=True)

In [174]:
test_df.head()

,displacement,time,stress
0,0.000000,0,0.000000
1,0.229787,4,22.674422
2,0.459574,8,45.214168
3,0.689362,12,67.831964
4,0.919149,16,90.332803


In [175]:
test_df.hvplot.scatter(y='stress',x='displacement')

ERROR:bokeh.core.validation.check:E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name. This could either be due to a misspelling or typo, or due to an expected column being missing. : key "fill_color" value "broken", key "line_color" value "broken" [renderer: GlyphRenderer(id='6954', ...)]


:Scatter   [displacement]   (stress)

In [176]:
test_df.hvplot.scatter(x='time',y='stress')

:Scatter   [time]   (stress)

In [150]:
test_df.hvplot(y='stress',x='displacement')

:Curve   [displacement]   (stress)

In [151]:
test_df.hvplot(y='stress',x='displacement')

:Curve   [displacement]   (stress)

In [152]:
Es = []
Ys = []
Hs = []
bs = []
test_dfs = []

for t in info_df.type:
    test_df, E_mod, Y_str, H, breaks = create_deformation_curve(t)
    Es.append(E_mod)
    Ys.append(Y_str)
    Hs.append(H)
    bs.append(breaks)
    test_dfs.append(test_df)


info_df['E-moduli'] = Es
info_df['Hardness'] = Hs
info_df['Strength'] = Ys
info_df['broke'] = bs


In [156]:
info_df.broke.count()

1000

In [166]:
conn = sqlite3.connect('TestDB.db')

In [167]:
info_df.to_sql('info',con=conn)

In [168]:
for i, df in enumerate(test_dfs):
    df.to_sql('test_{0}'.format(i),con=conn)

:Bars   [index,type]   (broke)